In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
# run this cell only the first time
Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.8"),
    PackageSpec(url="https://github.com/sadit/UMAP.jl", rev="54bcaa88a47a8597f1a3a390e495ec032066aa6b"),
    PackageSpec(name="Plots"),
    PackageSpec(name="Primes"),
    PackageSpec(name="StatsBase")
])


In [ ]:

using SimilaritySearch, UMAP, Primes, Plots, StatsBase

# Prime numbers
This demonstration is about prime numbers and its similarity based on its factors. This is a well-known demonstration of `UMAP.jl`. This notebook does not requires to download any dataset.



Now, we can define our dataset. The set of factors are found using the `Primes` package. Note that we use `VectorDatabase` to represent the dataset.

In [ ]:
n = 100_000
F = Vector{Vector{Int32}}(undef, n)

for i in 2:n+1
    s = Int32[convert(Int32, f) for f in factor(Set, i)]
    sort!(s)
    F[i-1] = s
end

db = VectorDatabase(F)

We use Int32 ordered arrays to store prime factors to represent each integer. In the following cell define the cosine distance equivalent for this representation. While other representations may perform faster, this is quite straighforward and demonstrates the use of user's defined distance functions.

In [ ]:

struct CosineDistanceSet <: SemiMetric end

function SimilaritySearch.evaluate(::CosineDistanceSet, U, V)
    _, isize = SimilaritySearch.union_intersection(U, V)
    1 - isize / (sqrt(length(U)) * sqrt(length(V)))
end



## Index construction
Note that the primes factors are pretty large for some large $n$ and this imply challengues for metric indexes (since it is related with the intrinsic dimension of the dataset). We used a kernel that starts 64 threads, it solves $100000$ in a few seconds but it can take pretty large time using single threads and larger $n$ values. The construction of the index is used by the visualization algorithm (UMAP) to construct an all-knn graph, which can be a quite costly procedure.

In [ ]:
#dist = JaccardDistance()  # Other distances from SimilaritySearch
dist = DiceDistance()
#dist = IntersectionDissimilarity()
#dist = CosineDistanceSet()
G = SearchGraph(; db, dist)
push!(G.callbacks, OptimizeParameters(kind=ParetoRecall()))
index!(G; parallel_block=1000)
IJulia.clear_output()

## Visualizing with UMAP projection
We select to initialize the embedding randomly, this could yield to low quality embeddings, but it is much faster than other techniques like spectral layout. Note that we pass the Search graph `G`. We also use a second call to compute a 3D embedding for computing a kind of colour embedding, here we pass `U2` to avoid recomputing several of the involved structures.

In [ ]:
U2 = UMAP_(G, 2; n_neighbors=15, init=:random, n_epochs=30)
U3 = UMAP_(U2, 3; init=:random, n_epochs=50)

In [ ]:
X = @view U2.embedding[1, :]
Y = @view U2.embedding[2, :]

function normcolors(V)
    min_, max_ = extrema(V)
    V .= (V .- min_) ./ (max_ - min_)
    V .= clamp.(V, 0, 1)
end

C = copy(U3.embedding)
normcolors(@view C[1, :])
normcolors(@view C[2, :])
normcolors(@view C[3, :])

C = [RGB(c...) for c in eachcol(C)]

scatter(X, Y, c=C, fmt=:png, size=(600, 600), ma=0.3, a=0.3, ms=1, msw=0, label="", yticks=nothing, xticks=nothing, title="First $n integers represented as prime factors\n $dist")
#savefig("umap-primes-1M.png")